## Import thư viện PyTorch

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## Đặt các siêu tham số (Hyper-parameters)

In [2]:
# Kịch thước đầu vào
input_size = 28 * 28    # 784

# Số lượng lớp
num_classes = 10

# Số lượng epochs
num_epochs = 5

# Kích thước batch
batch_size = 100

# Tốc độ học - Learning rate
learning_rate = 0.001

## Load bộ dữ liệu MNTST

In [3]:
# Dữ liệu tập huấn luyện
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

# Dữ liệu tập kiểm tra
test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

Failed to download (trying next):
HTTP Error 503: Service Unavailable



9913344it [00:14, 669864.51it/s]                                                


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



29696it [00:00, 96683.34it/s]                                                   


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



1649664it [00:02, 823877.17it/s]                                                


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



5120it [00:00, 1518837.01it/s]                                                  
/home/lenux/miniconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Processing...
Done!


## DataLoader

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Mô hình hồi quy Logistic (Logistic regression model)

In [5]:
model = nn.Linear(input_size, num_classes)

## Hàm mất mát (loss function) và bộ tối ưu hóa (optimizer)

In [6]:
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

## Huấn luyện mô hình

In [7]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Định dáng lại (reshape) dữ liệu thành (batch_size, input_size)
        images = images.reshape(-1, input_size)
        
        # Lan truyền tiến (Forward pass) và tính toán mất mát
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Lan truyền ngược (Backward pass) và tối ưu
        ## Đăt zero_grad để không tính toán đạo hàm
        optimizer.zero_grad()
        ## Gọi backward()
        loss.backward()
        ## Tối ưu
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2000
Epoch [1/5], Step [200/600], Loss: 2.1182
Epoch [1/5], Step [300/600], Loss: 2.0108
Epoch [1/5], Step [400/600], Loss: 1.9585
Epoch [1/5], Step [500/600], Loss: 1.8829
Epoch [1/5], Step [600/600], Loss: 1.7862
Epoch [2/5], Step [100/600], Loss: 1.7273
Epoch [2/5], Step [200/600], Loss: 1.6998
Epoch [2/5], Step [300/600], Loss: 1.6470
Epoch [2/5], Step [400/600], Loss: 1.5523
Epoch [2/5], Step [500/600], Loss: 1.4947
Epoch [2/5], Step [600/600], Loss: 1.4258
Epoch [3/5], Step [100/600], Loss: 1.3991
Epoch [3/5], Step [200/600], Loss: 1.3533
Epoch [3/5], Step [300/600], Loss: 1.3470
Epoch [3/5], Step [400/600], Loss: 1.2751
Epoch [3/5], Step [500/600], Loss: 1.2907
Epoch [3/5], Step [600/600], Loss: 1.2698
Epoch [4/5], Step [100/600], Loss: 1.1344
Epoch [4/5], Step [200/600], Loss: 1.1033
Epoch [4/5], Step [300/600], Loss: 1.1275
Epoch [4/5], Step [400/600], Loss: 1.1226
Epoch [4/5], Step [500/600], Loss: 1.1582
Epoch [4/5], Step [600/600], Loss:

## Kiểm tra mô hình

In [8]:
# Ở giai đoạn kiểm tra mô hình, không cần tính đạo hàm nên sử dụng with torch.no_grad()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 83.06999969482422 %


## Lưu mô hình (model checkpoint)

In [9]:
torch.save(model.state_dict(), 'model.ckpt')